In [1]:
from tl_tools import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
from tensorflow.keras.applications import ResNet50

In [2]:
setup_mixed_precision()
train_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/train'

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True ,vertical_flip=True)
train =train_datagen.flow_from_directory(train_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))


X_train, y_train, X_val, y_val = train_val_split(train, val_split= 0.3)

Mixed precision policy set to: mixed_float16

Found 4980 images belonging to 6 classes.
Number of batches in the training data: 312
Batch size of a single batch 16
Number of samples in the training dataset 4980

Number of training data batches with val split of 0.3: 219
Number of validation data batches: 93



2024-11-14 17:04:24.334433: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-11-14 17:04:24.334459: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-11-14 17:04:24.334465: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-11-14 17:04:24.334479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-14 17:04:24.334488: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Shape of image training set: (3504, 256, 256, 3)
Shape of image validation set: (1476, 256, 256, 3)

Shape of label training set: (3504, 6)
Shape of label validation set: (1476, 6)


In [3]:
pooling = 'avg'
resnet50 = ResNet50(weights= None ,include_top= False, input_shape= (256, 256, 3), pooling = pooling)

model = fc_layers(resnet50, name = 'resnet50v1')

In [4]:
model.compile(optimizer=Adam(learning_rate = 0.001, clipnorm= 1.0),
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=True, start_from_epoch=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

In [5]:
save_freq = (5 * 110) 

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/training/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:03d}.keras'),
    save_weights_only=False, 
    save_best_only=False,     
    save_freq= save_freq, #saves after every 10 * 110 batches per epoch (10 epochs), experienced memory issues
    verbose=1
)


#history =  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1, 
#                          callbacks= [early_stopping, reduce_lr, checkpoint_callback])

In [6]:
latest_checkpoint = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/training/checkpoints/model_epoch_085.keras'
model = tf.keras.models.load_model(latest_checkpoint)
lr = float(tf.keras.backend.get_value(model.optimizer.learning_rate))
print(f'Current learning rate: {lr}')

model.compile(
    optimizer = Adam(learning_rate= lr),
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy','F1Score']
)

history = model.fit(X_train, y_train, validation_data= (X_val, y_val), epochs = 120, initial_epoch= 85, 
                               callbacks = [checkpoint_callback, reduce_lr, early_stopping])

Current learning rate: 0.0001250000059371814
Epoch 86/120


2024-11-14 17:04:35.056889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


110/110 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - F1Score: 0.9947 - accuracy: 0.9948 - loss: 0.0294 - val_F1Score: 0.8331 - val_accuracy: 0.8354 - val_loss: 0.6392 - learning_rate: 1.2500e-04
Epoch 87/120
110/110 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - F1Score: 0.9915 - accuracy: 0.9916 - loss: 0.0304 - val_F1Score: 0.8224 - val_accuracy: 0.8232 - val_loss: 0.7696 - learning_rate: 1.2500e-04
Epoch 88/120
110/110 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - F1Score: 0.9912 - accuracy: 0.9913 - loss: 0.0346 - val_F1Score: 0.8243 - val_accuracy: 0.8279 - val_loss: 0.7358 - learning_rate: 1.2500e-04
Epoch 89/120
110/110 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - F1Score: 0.9929 - accuracy: 0.9929 - loss: 0.0336 - val_F1Score: 0.8467 - val_accuracy: 0.8503 - val_loss: 0.6458 - learning_rate: 1.2500e-04
Epoch 90/120
109/110 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - F1Score: 0.9893 - accuracy: 0.9895 - loss: 0.0383
Epoch 90: saving model to /Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/traini

In [7]:
path = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/trained_models'
model_save_path = os.path.join(path, 'resnet_50v1.keras')
model.save(model_save_path) 